In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_generator = ImageDataGenerator()

In [3]:
directory_generator = img_generator.flow_from_directory(
    directory = r"dataset\data_sample\train", # the folder where the class subfolders can be found
    target_size = (512,512), # The (height,width) of the produced images
    class_mode = "sparse", # Wether the target should be represented by an index or a dummy vector
    batch_size=16, # The batch size of the produced batches
    shuffle = True #Whether to shuffle after all files have been selected once
    #subset = "training"
)

directory_generator_val = img_generator.flow_from_directory(
    directory = r"dataset\data_sample\test", # the folder where the class subfolders can be found
    target_size = (512,512), # The (height,width) of the produced images
    class_mode = "sparse", # Wether the target should be represented by an index or a dummy vector
    batch_size=16, # The batch size of the produced batches
    shuffle = True #Whether to shuffle after all files have been selected once
    #subset = "validation"
)

Found 3472 images belonging to 4 classes.
Found 1162 images belonging to 4 classes.


In [4]:
imgs, targets = next(iter(directory_generator))

In [5]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

modelconv = tf.keras.Sequential(
    [
     Conv2D(
         filters = 32,
         kernel_size = (3,3),
         strides = 2,
         padding = "same",
         activation = "relu",
         input_shape = (imgs.shape[1], imgs.shape[2], imgs.shape[3]) # the input shape (height, width, channels)
     ),
     Conv2D(
         filters = 64,
         kernel_size = (3,3),
         strides = 2,
         padding = "same",
         activation = "relu"
     ),
     Conv2D(
         filters = 64,
         kernel_size = (3,3),
         strides = 2,
         padding = "same",
         activation = "relu"
     ),
     Conv2D(
         filters = 128,
         kernel_size = (3,3),
         strides = 2,
         padding = "same",
         activation = "relu"
     ),
    tf.keras.layers.Flatten(), # this layer turns multi-dimensional images into flat objects
    tf.keras.layers.Dense(24, activation="softmax")
    ]
)

c:\Users\Dominique Ourmiah\anaconda3\envs\tensorflow\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam

modelconv.compile(optimizer=Adam(),
              loss=SparseCategoricalCrossentropy(),
              metrics=[SparseCategoricalAccuracy()])

modelconv.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 24)             │     3,145,752 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,275,928 (12.50 MB)

 Trainable params: 3,275,928 (12.50 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
print(directory_generator.directory)

dataset\data_sample\train


In [8]:
modelconv.fit(directory_generator, epochs = 10, validation_data=directory_generator_val)

Epoch 1/10


c:\Users\Dominique Ourmiah\anaconda3\envs\tensorflow\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


217/217 ━━━━━━━━━━━━━━━━━━━━ 117s 532ms/step - loss: 17.6488 - sparse_categorical_accuracy: 0.3827 - val_loss: 1.5349 - val_sparse_categorical_accuracy: 0.4596
Epoch 2/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 119s 545ms/step - loss: 1.0617 - sparse_categorical_accuracy: 0.5628 - val_loss: 1.2129 - val_sparse_categorical_accuracy: 0.4131
Epoch 3/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 111s 510ms/step - loss: 1.0198 - sparse_categorical_accuracy: 0.6064 - val_loss: 1.0897 - val_sparse_categorical_accuracy: 0.5800
Epoch 4/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 108s 496ms/step - loss: 0.7682 - sparse_categorical_accuracy: 0.6994 - val_loss: 1.2335 - val_sparse_categorical_accuracy: 0.5387
Epoch 5/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 110s 504ms/step - loss: 0.6465 - sparse_categorical_accuracy: 0.7410 - val_loss: 1.2248 - val_sparse_categorical_accuracy: 0.6661
Epoch 6/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 111s 508ms/step - loss: 0.4919 - sparse_categorical_accuracy: 0.8163 - val_loss: 1.5532 - val_sparse_categorical_accurac

In [9]:
from plotly import graph_objects as go
color_chart = ["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]

history = modelconv.history
fig = go.Figure(data=[
                      go.Scatter(
                          y=history.history["loss"],
                          name="Training loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[0]
                          )),
                      go.Scatter(
                          y=history.history["val_loss"],
                          name="Validation loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title='Training and val loss across epochs',
    xaxis_title='epochs',
    yaxis_title='Cross Entropy'    
)
fig.show()